# Computing with word embeddings: Exercises

| Author | Last update |
|:------ |:----------- |
| Hauke Licht (https://github.com/haukelicht) | 2023-09-24 |

This notebook illustrates how to use `gensim` to compute with word vectors (e.g., word2vec) to, for example

- compute two words similarity
- find the most similar words for a focal word
- solve analogy tasks

## Setup

Load required modules.

In [1]:
# file in- and export
import os

# for working with word embeddings
import gensim
import gensim.downloader as api

# for using arrays and data frames
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
plt.style.use('ggplot')

Load a pre-trained word embedding model with `gensim`'s model API.

In [2]:
# load the model and name it's instance in our notebook environment 'word2vec'
word2vec = api.load('word2vec-google-news-300')

**_Note_:** You can also load another model if you want. It will still be a `KeyedVectors` object. So you can directly apply what you learned.

In [3]:
# list available models
print(list(api.info()['models'].keys()))

# get detailed info for a specific model
api.info(name='word2vec-google-news-300')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


{'num_records': 3000000,
 'file_size': 1743563840,
 'base_dataset': 'Google News (about 100 billion words)',
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/word2vec-google-news-300/__init__.py',
 'license': 'not found',
 'parameters': {'dimension': 300},
 'description': "Pre-trained vectors trained on a part of the Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. The phrases were obtained using a simple data-driven approach described in 'Distributed Representations of Words and Phrases and their Compositionality' (https://code.google.com/archive/p/word2vec/).",
 'read_more': ['https://code.google.com/archive/p/word2vec/',
  'https://arxiv.org/abs/1301.3781',
  'https://arxiv.org/abs/1310.4546',
  'https://www.microsoft.com/en-us/research/publication/linguistic-regularities-in-continuous-space-word-representations/?from=http%3A%2F%2Fresearch.microsoft.com%2Fpubs%2F189726%2Frvec

## 1. Word vector similarities

### Exercise 1

Use the `wordnet` python package to find synonyms and antonyms of your choice of focus word(s), and compute their similarities.
What do you observe?

**_Hint:_** You can also define your own lists of synonyms and antonyms.

**_Note:_** *WordNet* is a lexical database and semantic network of words and their relationships. 
It was developed to assist natural language processing and computational linguistics applications by providing a structured and comprehensive way to represent the English language vocabulary. 
WordNet was created at Princeton University and has been widely used in various text analysis tasks, including machine learning, information retrieval, and natural language understanding.
([source](https://chat.openai.com/share/1de49018-f487-4789-82cb-98ccf8d47ccb))

In [22]:
# import wordnet
from nltk.corpus import wordnet

# TODO: insert your focus word here
focus_word = 'nice' # <== change!

# find the sysnset for the focus word of your choice
synsets = wordnet.synsets(focus_word)
for synset in synsets:
    print(synset, synset.definition())

Synset('nice.n.01') a city in southeastern France on the Mediterranean; the leading resort on the French Riviera
Synset('nice.a.01') pleasant or pleasing or agreeable in nature or appearance; - George Meredith
Synset('decent.s.01') socially or conventionally correct; refined or virtuous
Synset('nice.s.03') done with delicacy and skill
Synset('dainty.s.04') excessively fastidious and easily disgusted
Synset('courteous.s.01') exhibiting courtesy and politeness


In [23]:
# find the synonyms and antonyms for <YOUR WORD>
synonyms = []
antonyms = []
for synset in wordnet.synsets(focus_word):
    for lemma in synset.lemmas():
        if focus_word not in lemma.name():
            synonyms.append(lemma.name())
        if lemma.antonyms():
            for a in lemma.antonyms():
                antonyms.append(a.name())

# print the results
print('Synonyms for: ' + focus_word)
print(set(synonyms))
print('Antonyms for: ' + focus_word)
print(set(antonyms))

Synonyms for: nice
{'prissy', 'squeamish', 'Nice', 'gracious', 'courteous', 'decent', 'skillful', 'dainty'}
Antonyms for: nice
{'nasty'}


In [24]:
# TODO: iterate over synonyms and antonyms and print the similarity between focus word and synonym/antonym pairs
for synonym in synonyms:
    if synonym in word2vec.index_to_key:
        print(f'"{synonym}": {word2vec.similarity(focus_word, synonym)}')

"Nice": 0.46746230125427246
"decent": 0.5993331670761108
"skillful": 0.20048925280570984
"dainty": 0.2785818874835968
"prissy": 0.25895628333091736
"squeamish": 0.19743596017360687
"courteous": 0.28391408920288086
"gracious": 0.4229739308357239


In [7]:
for antonym in antonyms:
    print(f'"{antonym}": {word2vec.similarity(focus_word, antonym)}')

"nasty": 0.3650338053703308


### EXERCISE 2

Let's implement a classic approach to evaluate how word embeddings capture cultural biases in their training copora.
Here, we'll focus on **_gender bias_** &mdash; the differential association of traits and attributes with women and men (my lose definition).

Compile a list `comparison_words` with occupations, character traits, and other words that might exhibit gender bias.
Then compute how similar each word is with terms like 'man' and 'women', that indicate the male and female genders.

Which words exhibit gender bias?
And in which direction? 
Do you spot a pattern?

In [27]:
comparison_words = [
    'programmer',
    'scientist',
    'smart',
    'emotional',
    'caring',
    'CEO',
    'annoying',
    'football',
    'soccer',
    'politician',
    'populist',
    'power',
    'nurse',
    'doctor'
    # add more interesting words here
]

In [31]:
# compute similarities to male and female terms
male_terms = ['man', 'men', 'male', 'he', 'him', 'his']
female_terms = ['woman', 'women', 'female', 'she', 'her', 'hers']

s_male = dict()
s_female = dict()

for word in comparison_words:
    s_male[word] = []
    s_female[word] = []
    for w in male_terms:
        s = word2vec.similarity(word, w)
        s_male[word].append(s)
    for w in female_terms:
        s = word2vec.similarity(word, w)
        s_female[word].append(s)

In [33]:
{w: np.mean(s) for w, s in s_female.items()}

{'programmer': 0.029597761,
 'scientist': 0.06876839,
 'smart': 0.041580766,
 'emotional': 0.18292125,
 'caring': 0.20474696,
 'CEO': -0.106728755,
 'annoying': 0.097587876,
 'football': 0.063607715,
 'soccer': 0.13928266,
 'politician': 0.22477663,
 'populist': 0.08099014,
 'power': 0.042475563,
 'nurse': 0.3203691,
 'doctor': 0.23384072}

In [30]:
{w: np.mean(s) for w, s in s_male.items()}

{'programmer': 0.054541085,
 'scientist': 0.10331989,
 'smart': 0.06168256,
 'emotional': 0.15299931,
 'caring': 0.17566599,
 'CEO': -0.0061667506,
 'annoying': 0.07397268,
 'football': 0.17734624,
 'soccer': 0.11919821,
 'politician': 0.22752604,
 'populist': 0.09036921,
 'power': 0.07486963,
 'nurse': 0.19163398,
 'doctor': 0.27489495}

In [ ]:
# TODO: summarize the results in a table or figure
# hint: if you have more than one term per gender, you might want to compute the average of comparison term--gender word similarities

### EXERCISE 3

Implement the same logic but now use the gender bisa-related word and attribute lists used in Caliskan *et al.*'s paper ["Semantics derived automatically from language corpora contain human-like biases"](https://www.science.org/doi/10.1126/science.aal4230).

You find the word and attribute lists in the folder `./../data/replications/caliskan_semantics_2017/wordlists/` (e.g., the file 'science_arts_male_female.txt')

## 2. Nearest neihbors

### Exercise 1

Let's use nearest neighbors search to find conceptually equivalent terms for a "seed" word.

**_Note:_** This is a typical task in expanding keyword lists for dictionaries.

You can choose which seed word you want to start with (see example below for a suggestion).
But while going through nearest neighbors, keep track of how many of the candidate terms in the top-20 or top-50 terms (or so) you would inlcude in your dictionary, and how many you would discard!

**_Example_**: 
Say you want to compile a dictionary that contains typical words used to express *positive emotions*.
In this case, you could start with the seed word 'happy.'

In [ ]:
[w for w, s in word2vec.most_similar('happy', topn=20)]

In [ ]:
# EXAMPLE: I made decisions
'glad'  # approved
'pleased', # approved
'ecstatic', # approved
'overjoyed', # approved
'thrilled', # approved
'satisfied', # approved
'proud', # approved
'delighted', # approved
'disappointed', # not approved <== !!!
'excited', # approved
'happier', # approved
'Said_Hirschbeck', # not approved <== !!!
# ... and so on

### Exercise 2

Discuss with your neighbor how one could improve this nearest neighbor search-based dictionary expansion strategy?
Do you ideas for automated quality checks?

## 3. Analogies

### Exercise 1

Can you come up with analogy problems involving terms from your discipline or research area?
Can the word embedding model solve these specialized problems?

**_Example:_** In politics "Democrat is to progressive what Republican is to ___?"

In [ ]:
v_x1 = word2vec['Democrat']
v_y1 = word2vec['progressive']
v_x2 = word2vec['Republican']

v_q = v_y1 - v_x1 + v_x2

word2vec.similar_by_vector(v_q, topn=5)

### Exercise 2

Take examples from one of the word lists in the folder `./../data/benchmarks/bats/3_encyclopedic_semantics/` to construct analgoy tests.
How well does the word2vec model perform on average?

**_Hint:_** Think about possible ways of defining performance.

In [ ]:
# example
fp = './../data/benchmarks/bats/3_encyclopedic_semantics/E01 [country - capital].txt'

with open(fp, 'r') as f:
    wordlist = [tuple(line.strip().split('\t')) for line in f]

wordlist[:10]

In [ ]:
# note: depending on your evaluation strategy, you might need to change this function
def analogy(x1='man', y1='king', x2='woman', verbose=True):
    """Computes return to query 'y1 is to x1 what WORD is to x2?'"""
    result = word2vec.most_similar(positive=[y1, x2], negative=[x1])
    if verbose:
        print(f"'{x1}' : '{y1}' :: '{x2}' : ?? ==> '{result[0][0]}'")
    return result[0][0]

In [ ]:
# analogy(x1='man', y1='king', x2='women') # :)
analogy(x1='abuja', y1='nigeria', x2='ankara') # :/
analogy(x1='nigeria', y1='abuja', x2='turkey') # x/
analogy(x1='athens', y1='greece', x2='baghdad') # :/
analogy(x1='berlin', y1='germany', x2='paris') # x/
analogy(x1='germany', y1='berlin', x2='france') # x/
analogy(x1='germany', y1='berlin', x2='france') # x/